# Building a web-scraping app by utilizing Gen-AI

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
#Langchain Tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
# Data Ingestion

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html")

loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html', 'title': 'UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\nUNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Main Content\n\n\n\n\n\n\n\n\n\n\n\nStudents |\n                \n\nFaculty & Staff |\n                \n\nAlumni |\n                \n\nCommunity |\n                \n\nGive to UNF |\n                \n\nmyWings\n\n\n\n\n\n\n\n\n\nA-Z Index\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                \n                2024-2025 University Catalog\n                \n                                    \n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n            CATALOG HOME\n        \n\n\nABOUT UNF\n\n\n\n            Introduction\n        \n\n\n\n            Mission & Vision\n        \n\n\n\n            Accreditations\n        \n\n\n\n 

In [6]:
# Data Transformation

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)


In [7]:
documents

[Document(metadata={'source': 'https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html', 'title': 'UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)', 'language': 'en'}, page_content='UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Main Content\n\n\n\n\n\n\n\n\n\n\n\nStudents |\n                \n\nFaculty & Staff |\n                \n\nAlumni |\n                \n\nCommunity |\n                \n\nGive to UNF |\n                \n\nmyWings\n\n\n\n\n\n\n\n\n\nA-Z Index\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                \n                2024-2025 University Catalog\n                \n                                    \n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n            CATALOG HOME\n        \n\n\nABOUT UNF\n\n\n\n            Introduction\n        \n\n\n\n            Mission & Vision\n        \n\n\n\n            Accreditations\n        \n\n\n\n            Campu

In [8]:
# Data Embedding

In [9]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [10]:
# Vectorstore DB

from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [11]:
vectorstoredb

In [14]:
# Query fromm a vector DB

query="enrollments may be allowed to fulfill the thesis requirements but only"

result=vectorstoredb.similarity_search_with_score(query)
result

[(Document(id='17b35ce2-9310-46a6-8b9f-b2cfae452a94', metadata={'source': 'https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html', 'title': 'UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)', 'language': 'en'}, page_content='The student conducts an in-depth study of a research problem leading to a Masters Thesis that summarizes the student findings. The student defends the Thesis in a presentation that is open to the general public and evaluated by a faculty committee.\nCIS6970 Thesis: Computer Science (3 Credits)'),
  np.float32(0.36952138)),
 (Document(id='6e9e2a7f-3e18-4e1f-88a6-bfadb243880d', metadata={'source': 'https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html', 'title': 'UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)', 'language': 'en'}, page_content='Core Requirements (9 credits)\nCIS6913 Research Methods in Computing (3 Credits)CIS6372 Information Assurance (3 Credits)CEN6940 Computing Practi

In [1]:
# retreival and document chain
# create_stufff_document_chain provides context for the model so that its answer is accurate, we send the document as a context

In [12]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [16]:
# Document Chain 

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain



RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023B266966D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023B38E57110>, root_client=<openai.OpenAI object at 0x0000023B29541FD0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023B2944FED0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'r

In [23]:
from langchain_core.documents import Document
document_chain.invoke(
    {
        "input":"enrollments may be allowed to fulfill the thesis requirements but only",
        "context":[Document(page_content="Enroll in CIS 6970 Thesis for at least two terms. More enrollments may be allowed to fulfill the thesis requirements, but only six credits are applicable to the degree. In order to enroll in the Thesis course, the student must obtain the approval of the Graduate Director and a Thesis Advisor.")]
    }
)

'How many credits from the CIS 6970 Thesis course can be applied to the degree?'

In [18]:
# We want the docs to come from retriever
# Input -> Retriever-> vectorstoredb

vectorstoredb

In [19]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain= create_retrieval_chain(retriever,document_chain)


In [ ]:
# Retriever is an interface with which we send an input and get the response from the vector DB

In [20]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023B297D4510>, search_kwargs={})

In [21]:
# Response from retriever

response=retrieval_chain.invoke({"input":"enrollments may be allowed to fulfill the thesis requirements but only"})
response['answer']

'What are the requirements for enrolling in the CIS6970 Thesis course?\n\nTo enroll in the CIS6970 Thesis course, the student must obtain the approval of the Graduate Director and a Thesis Advisor. Additionally, the program specifies that the student needs to enroll in the thesis course for at least two terms, contributing to the overall thesis requirement of 6 credits applicable to the degree.'

In [24]:
response['context']

[Document(id='caad7e7a-33aa-4f53-813a-b54a4696e6d6', metadata={'source': 'https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html', 'title': 'UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)', 'language': 'en'}, page_content='The student conducts an in-depth study of a research problem leading to a Masters Thesis that summarizes the student findings. The student defends the Thesis in a presentation that is open to the general public and evaluated by a faculty committee.\nCIS6970 Thesis: Computer Science (3 Credits)'),
 Document(id='85e5edf8-a88a-4e5d-951b-90a2896bb90f', metadata={'source': 'https://www.unf.edu/catalog/programs/gr/ccec/CCEC-MS-30CCISCISJ.html', 'title': 'UNF: Computing & Info Sciences: Data Science (Thesis) (Master of Science)', 'language': 'en'}, page_content='Core Requirements (9 credits)\nCIS6913 Research Methods in Computing (3 Credits)CIS6372 Information Assurance (3 Credits)CEN6940 Computing Practicum (3 Credits)\nConcentratio